In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown --id 1JIpccI_hh8L5Rr_yf81cSXA3__K0fXVn

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1JIpccI_hh8L5Rr_yf81cSXA3__K0fXVn
From (redirected): https://drive.google.com/uc?id=1JIpccI_hh8L5Rr_yf81cSXA3__K0fXVn&confirm=t&uuid=d9c00e2f-cc7d-49f4-8962-157cadd4b523
To: /content/dataset_label.zip
100% 3.25G/3.25G [01:04<00:00, 50.7MB/s]


In [ ]:
!unzip /content/dataset_label.zip -d /content

Archive:  /content/dataset_label.zip
   creating: /content/dataset_label/
   creating: /content/dataset_label/0/
   creating: /content/dataset_label/90/
   creating: /content/dataset_label/70/
   creating: /content/dataset_label/60/
   creating: /content/dataset_label/10/
   creating: /content/dataset_label/30/
   creating: /content/dataset_label/20/
   creating: /content/dataset_label/40/
   creating: /content/dataset_label/50/
   creating: /content/dataset_label/80/
   creating: /content/dataset_label/100/
   creating: /content/dataset_label/230/
   creating: /content/dataset_label/250/
   creating: /content/dataset_label/160/
   creating: /content/dataset_label/270/
   creating: /content/dataset_label/180/
   creating: /content/dataset_label/150/
   creating: /content/dataset_label/340/
   creating: /content/dataset_label/210/
   creating: /content/dataset_label/190/
   creating: /content/dataset_label/220/
   creating: /content/dataset_label/300/
   creating: /content/dataset_label

In [ ]:
# !unzip /content/drive/MyDrive/PROJECT\ SDP/dataset_label.zip -d /content

In [ ]:
!pip install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 54.7 MB/s eta 0:00:00


In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
from timm import create_model
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import os
from PIL import Image

In [ ]:
# Define the dataset
class BottleRotationDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform
        for degree_folder in os.listdir(root_dir):
            folder_path = os.path.join(root_dir, degree_folder)
            if os.path.isdir(folder_path):
                degree = float(degree_folder)
                for img_name in os.listdir(folder_path):
                    self.image_paths.append(os.path.join(folder_path, img_name))
                    self.labels.append(degree)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.float32)

In [ ]:
# Define the custom PyTorch Lightning model
class ViTRegression(pl.LightningModule):
    def __init__(self, learning_rate=1e-4):
        super().__init__()
        self.save_hyperparameters()
        self.vit = create_model('vit_tiny_patch16_224', pretrained=True)
        num_features = self.vit.head.in_features
        self.vit.head = nn.Identity()  # Remove classification head
        self.regressor = nn.Linear(num_features, 1)  # Add regression head
        self.criterion = nn.MSELoss()

    def forward(self, x):
        x = self.vit(x)
        x = self.regressor(x)
        return x

    def training_step(self, batch, batch_idx):
        images, targets = batch  # Targets are now in the range [0, 360]
        outputs = self(images).squeeze()  # Predictions from the model
        loss = self.criterion(outputs, targets)  # Compute loss without normalization

        # Log the learning rate
        current_lr = self.optimizers().param_groups[0]['lr']
        self.log('learning_rate', current_lr, on_step=False, on_epoch=True)

        # Log only at the end of the epoch
        self.log('train_loss', loss, on_step=False, on_epoch=True)

        return loss

    def configure_optimizers(self):
        # Define default values
        default_learning_rate = 1e-4
        default_scheduler_patience = 5
        default_scheduler_factor = 0.1
        default_scheduler_threshold = 100  # Adjust to match your loss scale
        default_min_lr = 1e-6

        # Retrieve hyperparameters or use defaults
        learning_rate = self.hparams.get('learning_rate', default_learning_rate)
        scheduler_patience = self.hparams.get('scheduler_patience', default_scheduler_patience)
        scheduler_factor = self.hparams.get('scheduler_factor', default_scheduler_factor)
        scheduler_threshold = self.hparams.get('scheduler_threshold', default_scheduler_threshold)
        min_lr = self.hparams.get('min_lr', default_min_lr)

        # Define optimizer
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)

        # Define scheduler
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',                  # Minimize the monitored metric
            factor=scheduler_factor,     # Reduce learning rate by this factor
            patience=scheduler_patience, # Wait this many epochs with no improvement
            threshold=scheduler_threshold,  # Minimum improvement to consider
            threshold_mode='abs',        # Use absolute threshold for large-scale losses
            cooldown=0,                  # No cooldown period after reduction
            min_lr=min_lr                # Minimum learning rate allowed
        )

        # Return optimizer and scheduler
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'train_loss',  # Metric to monitor for learning rate adjustment
                'interval': 'epoch',      # Check at the end of every epoch
                'frequency': 1            # Frequency of scheduler updates
            }
        }

In [ ]:
# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Dataset and DataLoader
root_dir = "/content/dataset_label"  # Replace with your dataset path
dataset = BottleRotationDataset(root_dir=root_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Lightning Trainer with Model Checkpoint Callback
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath="checkpoints/",
    filename="vit_tiny_rotation_epoch_{epoch}",
    save_top_k=-1,  # Save all checkpoints
    every_n_epochs=5,  # Save every 5 epochs
)

from pytorch_lightning.callbacks import ProgressBar

# Custom ProgressBar to ensure logging at the end of the epoch
class CustomProgressBar(ProgressBar):
    def __init__(self):
        super().__init__()

    def on_train_epoch_end(self, trainer, pl_module):
        train_loss = trainer.callback_metrics.get("train_loss", None)
        learning_rate = trainer.callback_metrics.get("learning_rate", None)
        if train_loss:
            print(f"Epoch {trainer.current_epoch + 1}: Train Loss = {train_loss:.4f}")

        if learning_rate:
            print(f"Epoch {trainer.current_epoch + 1}: Learning Rate: {learning_rate:.6f}")


# Trainer configuration
trainer = pl.Trainer(
    max_epochs=55,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    callbacks=[checkpoint_callback, CustomProgressBar()],  # Add the custom ProgressBar
)

# Initialize and Train
model = ViTRegression.load_from_checkpoint("/content/vit_tiny_rotation_epoch_epoch=90.ckpt")
trainer.fit(model, dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | vit       | VisionTransformer | 5.5 M  | train
1 | regressor | Linear            | 193    | train
2 | criterion | MSELoss           | 0      | train
--------------------------------------------------------
5.5 M     Trainable params
0         Non-trainable params
5.5 M     Total params
22.098    Total estimated model params size (MB)
266       Modules in train mode
0         Modules in eval mode
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (40) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 1: Train Loss = 129.2978
Epoch 1: Learning Rate: 0.000100
Epoch 2: Train Loss = 82.1581
Epoch 2: Learning Rate: 0.000100
Epoch 3: Train Loss = 65.2268
Epoch 3: Learning Rate: 0.000100
Epoch 4: Train Loss = 53.6707
Epoch 4: Learning Rate: 0.000100
Epoch 5: Train Loss = 46.1489
Epoch 5: Learning Rate: 0.000100
Epoch 6: Train Loss = 36.4229
Epoch 6: Learning Rate: 0.000100
Epoch 7: Train Loss = 30.7408
Epoch 7: Learning Rate: 0.000100
Epoch 8: Train Loss = 31.9539
Epoch 8: Learning Rate: 0.000010
Epoch 9: Train Loss = 26.0191
Epoch 9: Learning Rate: 0.000010
Epoch 10: Train Loss = 24.8773
Epoch 10: Learning Rate: 0.000010
Epoch 11: Train Loss = 24.2770
Epoch 11: Learning Rate: 0.000010
Epoch 12: Train Loss = 23.5259
Epoch 12: Learning Rate: 0.000010
Epoch 13: Train Loss = 22.9539
Epoch 13: Learning Rate: 0.000010


INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
### EVALUATION CODE

In [ ]:
!gdown --id 13R5JajJ4oh_enkRjfs2fThITGsIMlFX8

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=13R5JajJ4oh_enkRjfs2fThITGsIMlFX8
From (redirected): https://drive.google.com/uc?id=13R5JajJ4oh_enkRjfs2fThITGsIMlFX8&confirm=t&uuid=813237fe-50fe-45e8-92c7-f731bf353788
To: /content/test_set.zip
100% 224M/224M [00:03<00:00, 67.9MB/s]


In [ ]:
!unzip /content/test_set.zip -d /content

Archive:  /content/test_set.zip
   creating: /content/test_set/
   creating: /content/test_set/0/
   creating: /content/test_set/70/
   creating: /content/test_set/60/
   creating: /content/test_set/90/
   creating: /content/test_set/10/
   creating: /content/test_set/20/
   creating: /content/test_set/30/
   creating: /content/test_set/40/
   creating: /content/test_set/50/
   creating: /content/test_set/80/
   creating: /content/test_set/100/
   creating: /content/test_set/230/
   creating: /content/test_set/250/
   creating: /content/test_set/160/
   creating: /content/test_set/180/
   creating: /content/test_set/270/
   creating: /content/test_set/150/
   creating: /content/test_set/190/
   creating: /content/test_set/210/
   creating: /content/test_set/220/
   creating: /content/test_set/340/
   creating: /content/test_set/300/
   creating: /content/test_set/140/
   creating: /content/test_set/240/
   creating: /content/test_set/350/
   creating: /content/test_set/320/
   creating

In [ ]:
# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Dataset and DataLoader
root_dir = "/content/test_set"  # Replace with your dataset path
dataset = BottleRotationDataset(root_dir=root_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# Load a saved checkpoint
model = ViTRegression.load_from_checkpoint("/content/vit_tiny_rotation_epoch_epoch=100.ckpt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

In [ ]:
import os
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from PIL import Image, ImageDraw, ImageFont

# Ensure the output directory exists
output_dir = "/content/predictions"
os.makedirs(output_dir, exist_ok=True)

# Function to make predictions, save images with labels, and return predictions and targets
def predict_and_save_images(model, dataloader, output_dir):
    model.eval()  # Set the model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_predictions = []  # Store all predictions
    all_targets = []      # Store all corresponding targets

    # Iterate through the DataLoader
    for batch_idx, (images, labels) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)
        with torch.no_grad():
            predictions = model(images).squeeze()  # Get model predictions

        # Append predictions and targets to the lists
        all_predictions.extend(predictions.cpu().tolist())
        all_targets.extend(labels.cpu().tolist())

        # Process each image in the batch
        for i in range(len(images)):
            # Convert the image back to [0, 1] range
            image = images[i].cpu().permute(1, 2, 0).numpy()  # Convert to (H, W, C)
            image = (image * 0.5 + 0.5) * 255  # Reverse normalization and scale to 0-255
            image = Image.fromarray(image.astype("uint8"))

            # Create a new canvas with extra space above the image
            new_height = image.height + 50  # Add 50px for text
            canvas = Image.new("RGB", (image.width, new_height), "white")  # White background
            canvas.paste(image, (0, 50))  # Paste the image below the text area

            # Draw real and predicted labels on the canvas
            draw = ImageDraw.Draw(canvas)
            font = ImageFont.load_default()  # Use a default font
            real_label = f"Real: {labels[i].item():.2f}"
            predicted_label = f"Pred: {predictions[i].item():.2f}"

            # Write the labels with different colors
            draw.text((10, 10), real_label, fill="red", font=font)  # Real label in red
            draw.text((10, 30), predicted_label, fill="blue", font=font)  # Predicted label in blue

            # Save the annotated image
            canvas.save(os.path.join(output_dir, f"image_{batch_idx}_{i}.png"))

    print(f"Annotated images saved to: {output_dir}")
    return all_predictions, all_targets  # Return all predictions and targets

# Run predictions, save images, and get predictions/targets
predictions, targets = predict_and_save_images(model, dataloader, output_dir)

# Print a few predictions and targets for verification
for i in range(5):  # Display the first 5 predictions and targets
    print(f"Target: {targets[i]:.2f}, Prediction: {predictions[i]:.2f}")

Annotated images saved to: /content/predictions
Target: 320.00, Prediction: 325.32
Target: 180.00, Prediction: 178.74
Target: 30.00, Prediction: 22.28
Target: 200.00, Prediction: 197.62
Target: 180.00, Prediction: 177.06


In [ ]:
import numpy as np

def compute_metrics(predictions, targets):
    # Convert to numpy arrays
    predictions = np.array(predictions)
    targets = np.array(targets)

    # MAE
    mae = np.mean(np.abs(predictions - targets))

    # MSE
    mse = np.mean((predictions - targets) ** 2)

    # RMSE
    rmse = np.sqrt(mse)

    # MAPE
    mape = np.mean(np.abs((targets - predictions) / (targets + 1e-8))) * 100  # Avoid division by zero

    # R-squared
    target_mean = np.mean(targets)
    r2 = 1 - np.sum((targets - predictions) ** 2) / np.sum((targets - target_mean) ** 2)

    # Angular Error
    angular_error = np.mean(np.minimum(
        np.abs(targets - predictions),
        360 - np.abs(targets - predictions)
    ))

    return {
        "Mean Absolute Error": mae,
        "Root Mean Squared Error": rmse,
    }

metrics = compute_metrics(predictions, targets)

for metric, value in metrics.items():
    print(f"{metric}: {value:.2f}")


"""

	•	MAE averages the errors linearly.
	•	RMSE grows quadratically due to squaring, amplifying the effect of the larger error.

"""

Mean Absolute Error: 5.04
Root Mean Squared Error: 14.28


'\n\n\t•\tMAE averages the errors linearly.\n\t•\tRMSE grows quadratically due to squaring, amplifying the effect of the larger error.\n\n'

In [ ]:
!zip -r /content/test_set_predictions.zip /content/predictions

updating: content/predictions/ (stored 0%)
updating: content/predictions/image_3_6.png (deflated 0%)
updating: content/predictions/image_0_22.png (deflated 0%)
updating: content/predictions/image_3_8.png (deflated 0%)
updating: content/predictions/image_1_2.png (deflated 0%)
updating: content/predictions/image_1_4.png (deflated 0%)
updating: content/predictions/image_2_5.png (deflated 0%)
updating: content/predictions/image_2_12.png (deflated 0%)
updating: content/predictions/image_0_5.png (deflated 0%)
updating: content/predictions/image_1_28.png (deflated 0%)
updating: content/predictions/image_1_13.png (deflated 0%)
updating: content/predictions/image_0_1.png (deflated 0%)
updating: content/predictions/image_2_22.png (deflated 0%)
updating: content/predictions/image_0_17.png (deflated 0%)
updating: content/predictions/image_0_23.png (deflated 0%)
updating: content/predictions/image_1_5.png (deflated 0%)
updating: content/predictions/image_1_22.png (deflated 0%)
updating: content/pre